In [2]:
import os
import pandas as pd
import numpy as np

import sys
sys.path.insert(0,'..')

import util


In [3]:

def load_data(data_dir: str, ds_set: str, lines: bool):

  ds_path = os.path.join(data_dir, ds_set + '.json')
  assert os.path.exists(ds_path), f"{ds_path} doesn't exsits"

  ds = pd.read_json(ds_path, lines=lines)
  ds['instructions'] = ds['content']
  return ds

In [4]:
 DATA_DIR = "/mnt/hackney/human_data"
 train = load_data(DATA_DIR, 'train', lines=True)
 valid = load_data(DATA_DIR, 'dev', lines=True)
 test = load_data(DATA_DIR, 'test', lines=True)


In [10]:
data_dir_geo = "/home/onlp_gcp_biu/cabby/app"

human_dataset = train
human_dataset['landmarks'] = ''


for human_dataset, data_type in zip([train, test, valid], ['train', 'dev', 'test']):
  geo_datasets_paths = list(set(human_dataset.rvs_path.tolist()))

  for geo_dataset_path in geo_datasets_paths:

    path_geo = geo_dataset_path.replace("/app_instructor", data_dir_geo)
    entities = util.load_entities(path_geo)

    keys = human_dataset[human_dataset['rvs_path']==geo_dataset_path].rvs_sample_number.tolist()
    entity_by_keys = np.array(entities)[keys]
    entity_landmarks_list = []
    for entity, key in zip(entity_by_keys, keys):
      landmarks = entity.geo_landmarks
      landmark_points = [f"{t}: {util.list_yx_from_point(v.geometry)}" for t, v in landmarks.items() if v.geometry]
      landmark_str = ';'.join(landmark_points)
      human_dataset.loc[human_dataset.rvs_sample_number== key, 'landmarks'] = landmark_str
      human_dataset.loc[human_dataset.rvs_sample_number== key, 'route'] = str(entity.route)



  new_dir = os.path.join(DATA_DIR, 'landmarks_data')
  if not os.path.exists(new_dir):
    os.mkdir(new_dir)
  new_path = os.path.join(new_dir, data_type + '.json')

  print (f'Saved {data_type} to: {new_path}')

  human_dataset.to_json(new_path, lines=True, orient='records')


Saved train to: /mnt/hackney/human_data/landmarks_data/train2.json
Saved dev to: /mnt/hackney/human_data/landmarks_data/dev2.json
Saved test to: /mnt/hackney/human_data/landmarks_data/test2.json
